In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [3]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-08-27
yesterday: 2025-08-26


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [9]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [11]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict)

In [13]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict)

In [15]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    return df_stocks.style.format(format_dict)

In [17]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [19]:
def update_dividend(const, name, new_qty):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [21]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'new_unit_cost': new_unit_cost, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

### End of Definition

## Begin of Sale transaction

In [23]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
df_sells_latest = pd.read_sql(sql, conpf)
df_sells_latest.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,MCS,43,2025-02-21,"6,000",7.30,1110,1316,2025-08-27,8.40,104.33,7.30,"50,288.37",187,"6,391.35",14.56%,28.42%,-110,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [25]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = df_sells_latest['name']
name = names.to_string(index=False)

sr_qty = df_sells_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = df_sells_latest['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = df_sells_latest['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = df_sells_latest['buy_id']
buy_id = buy_ids.to_string(index=False)
print(name, qty, buy_price, sell_price)

MCS -6000 7.3 8.4


### Begin of Update buy table in MySQL stock database from sale transaction

In [144]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.format(format_dict)

,name,date,shares,price,active,period,grade
0,MCS,2016-09-20,"75,000",15.31,1,2,A4


In [150]:
old_qty = int(df_buy['shares'].iloc[0])
old_price = df_buy['price'].iloc[0]
old_qty, old_price 

(75000, 15.312)

In [169]:
old_cost_amt = old_qty * old_price
sales_amt = qty * sell_price
new_qty = old_qty + qty
new_cost_amt = old_cost_amt + sales_amt
new_unit_cost = new_cost_amt / new_qty
print(name, old_cost_amt, sales_amt, new_cost_amt, new_qty, new_unit_cost)

MCS 1148400.0 -50400.0 1098000.0 69000 15.91304347826087


In [33]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,MCS,2016-09-20,"81,000",14.80,1,2,A4


In [171]:
print(name, new_qty, new_unit_cost)

MCS 69000 15.91304347826087


In [81]:
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [164]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,MCS,2016-09-20,"75,000",15.31,1,2,A4


In [87]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0 AND active = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [93]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,price,percent,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2500,0.0000,0.7500,0.00,0.00%,"81,000",2025-08-26,2025-09-10,,1


In [95]:
new_qty = df_dividend['shares'].iloc[0] + qty
new_qty

75000

In [97]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2500,0.0000,0.7500,"81,000",2025-08-26,2025-09-10,,1


In [99]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [101]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2500,0.0000,0.7500,"75,000",2025-08-26,2025-09-10,,1


In [103]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0 AND actual = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


In [105]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2500,0.0000,0.7500,"75,000",2025-08-26,2025-09-10,,1


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [109]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

,name,cost,available_qty
0,MCS,14.8,81000


In [111]:
old_qty = int(df_stocks['available_qty'].iloc[0])
new_qty = old_qty + qty
new_buy_target = 8.00
new_sell_target = 8.80
new_buy_qty = 5000

In [113]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.00,0.00,S,6.30,8.75,0.00,0.00,14.80,"6,000",-4,4,"81,000",0.000000,0.000000,20Pct,SET


In [121]:
print(name, transaction, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

MCS S 75000 15.312 8.0 8.8 5000


In [123]:
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [125]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.00,0.00,S,8.00,8.80,0.00,0.00,15.31,"5,000",-4,4,"75,000",0.000000,0.000000,20Pct,SET


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

In [130]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' \
ORDER BY period, buy.name".format(yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-08-26' ORDER BY period, buy.name


In [132]:
df_buy = pd.read_sql(sql, const)
df_buy

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,15.00,"44,400.00","18,000.00","-26,400.00",-59.46
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.38,"44,640.00","21,420.00","-23,220.00",-52.02
2,1,A2,JMART,2023-01-20,"6,800",33.00,8.55,"224,400.00","58,140.00","-166,260.00",-74.09
3,1,C2,ORI,2022-04-07,"50,000",9.00,2.24,"450,000.00","112,000.00","-338,000.00",-75.11
4,1,C2,PTTGC,2021-03-17,"6,000",64.75,27.00,"388,500.00","162,000.00","-226,500.00",-58.30
5,1,C2,SCC,2021-09-20,600,405.00,219.00,"243,000.00","131,400.00","-111,600.00",-45.93
6,1,A3,SINGER,2023-01-19,"6,000",24.80,5.55,"148,800.00","33,300.00","-115,500.00",-77.62
7,2,A1,AIMIRT,2023-08-17,"17,500",10.60,9.70,"185,500.00","169,750.00","-15,750.00",-8.49
8,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.40,"990,000.00","627,000.00","-363,000.00",-36.67
9,2,C1,DIF,2020-08-01,"50,000",12.17,8.20,"608,500.00","410,000.00","-198,500.00",-32.62


In [134]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [136]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [138]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [140]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)
df_buy.to_excel(xsl_file, index=False)